<a href="https://colab.research.google.com/github/gibranfp/CursoDatosMasivosI/blob/main/notebooks/4d_momentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo de Alon-Matias-Szegedy
En esta libreta programaremos el algoritmo de Alon-Matias-Szegedy (AMS) para estimar momentos en un flujo de datos.

Dado un flujo de tamaño $n$ constante, se toman $K$ variables $X_1, X_2, \ldots, X_K$ de forma aleatoria y uniforme. Estas variables almacenan un elemento $X_k.elemento$ y un valor entero $X_k.valor$, el cual se inicializa con 1 y se incrementa en 1 cada vez que se encuentra una ocurrencia de $X_k.elemento$.
 
De esta forma, es posible estimar el $i$-ésimo momento a partir de una variable $X_k$ calculando

$$
n \cdot (X_k.valor^{i} - (X_k.valor - 1)^{i})
$$

In [1]:
from collections import Counter
import numpy as np
np.random.seed(2021) # para reproducibilidad

n = 100
x_min = 0
x_max = 10
muestras = 50

Definimos la clase para el algoritmo AMS

In [2]:
class AMS:  
  def __init__(self, n_muestras=10):
    self.n_muestras = n_muestras

  def estima_momento(self, k):
    return np.mean(self.n * (self.valores**k - (self.valores - 1)**k))

  def calcula_cuentas(self, x):
    self.n = x.shape[0]
    self.ind = np.random.randint(0, self.n - 1, size=self.n_muestras)
    self.elementos = x[self.ind]
    self.valores = np.zeros_like(self.elementos)
    for i,ind in enumerate(self.ind):
      for j in range(ind, self.n):
        if self.elementos[i] == x[j]:
          self.valores[i] += 1

El momento $i$-ésimo está definido por
$$
\sum_e (m_e)^i
$$

donde $m_e$ es el número de veces que ocurre el elemento $e$ en el flujo.

In [3]:
momento = lambda m, i: np.sum(m**i)

Generamos un flujo de números enteros aleatorios.

In [4]:
flujo = np.random.randint(x_min, x_max, size=n)
frec = np.array(list(Counter(flujo).values()))

Instaciamos nuestra clase, calculamos las cuentas del elemento correspondiente a cada muestra y estimamos los momentos 1, 2 y 3.

In [5]:
em = AMS(muestras)
em.calcula_cuentas(flujo)

for k in range(1, 4):
  print(u'Momento {0}: Exacto = {1} Estimación = {2}'.format(k, 
                                                             momento(frec, k), 
                                                             em.estima_momento(k)))

Momento 1: Exacto = 100 Estimación = 100.0
Momento 2: Exacto = 1100 Estimación = 1128.0
Momento 3: Exacto = 12790 Estimación = 13444.0


Cuando el tamaño del flujo no es constante, seleccionamos las posiciones de las variables de la siguiente manera:
+ Se toman las primeras $s$ posiciones del flujo como variables.
+ Se elige la posición $n>s$ con probabilidad $\frac{s}{n}$
  + Si es elegida, se selecciona de forma aleatoria y uniforme una de las $s$ variables y se reemplaza por la de la posición $n$
  + En caso contrario se mantienen las posiciones de las $s$ variables    

In [6]:
class AMSFlujo:
  def __init__(self, n_muestras):
    self.n_muestras = n_muestras
    self.i = 0
    self.elementos = np.zeros(self.n_muestras)
    self.valores = np.zeros(self.n_muestras)

  def estima_momento(self, k):
    if self.i >= self.n_muestras:
      return np.mean(self.i * (self.valores**k - (self.valores - 1)**k))
    else:
      return np.mean(self.i * (self.valores[:self.i]**k - (self.valores[:self.i] - 1)**k))
    
  def __call__(self, x):
    if self.i < self.n_muestras:
      self.elementos[self.i] = x
      self.valores[self.i] = 0
      for a,e in enumerate(self.elementos[:self.i+1]):
        if e == x:
          self.valores[a] += 1
    else:
      prob = self.n_muestras / (self.i + 1)
      j = np.random.choice([0, 1], p=[1 - prob, prob])
      if j:
        pos = np.random.randint(0, self.n_muestras)
        self.elementos[pos] = x
        self.valores[pos] = 0
      
      for a,e in enumerate(self.elementos):
        if e == x:
          self.valores[a] += 1

    self.i += 1

Instanciamos la clase y vamos agregando cada dato del flujo, actualizando las cuentas y estimando los momentos 1, 2 y 3.

In [7]:
emf = AMSFlujo(muestras)
for i in range(n):
  emf(flujo[i])
  frec = np.array(list(Counter(flujo[:i+1]).values()))
  print(u'Posición {0}'.format(i))
  for k in range(1, 4):
    print(u'\tMomento {0}: Exacto = {1} Estimación = {2}'.format(k, 
                                                                 momento(frec, k), 
                                                                 emf.estima_momento(k)))

Posición 0
	Momento 1: Exacto = 1 Estimación = 1.0
	Momento 2: Exacto = 1 Estimación = 1.0
	Momento 3: Exacto = 1 Estimación = 1.0
Posición 1
	Momento 1: Exacto = 2 Estimación = 2.0
	Momento 2: Exacto = 2 Estimación = 2.0
	Momento 3: Exacto = 2 Estimación = 2.0
Posición 2
	Momento 1: Exacto = 3 Estimación = 3.0
	Momento 2: Exacto = 3 Estimación = 3.0
	Momento 3: Exacto = 3 Estimación = 3.0
Posición 3
	Momento 1: Exacto = 4 Estimación = 4.0
	Momento 2: Exacto = 4 Estimación = 4.0
	Momento 3: Exacto = 4 Estimación = 4.0
Posición 4
	Momento 1: Exacto = 5 Estimación = 5.0
	Momento 2: Exacto = 5 Estimación = 5.0
	Momento 3: Exacto = 5 Estimación = 5.0
Posición 5
	Momento 1: Exacto = 6 Estimación = 6.0
	Momento 2: Exacto = 8 Estimación = 8.0
	Momento 3: Exacto = 12 Estimación = 12.0
Posición 6
	Momento 1: Exacto = 7 Estimación = 7.0
	Momento 2: Exacto = 9 Estimación = 9.0
	Momento 3: Exacto = 13 Estimación = 13.0
Posición 7
	Momento 1: Exacto = 8 Estimación = 8.0
	Momento 2: Exacto = 12 Esti